In [1]:
from datetime import date

import numpy as np

import pandas as pd
import pandas_datareader.data as web
import pandas_datareader as pdr

import yfinance as yf
import quandl

In [2]:
STARTDATE = '1992-01-01'
ENDDATE = '2021-11-30'

In [3]:
# tickers = ['SPY','USO','TLT','UUP','TIP','LQD','GLD','IWN']
# df = yf.download(tickers, start=STARTDATE, end=ENDDATE)['Adj Close']

# prices = df.resample('M').last()
# rets_raw = prices.pct_change().dropna()

In [4]:
file_key = open("../dev/archive/quandl_key.txt","r")
API_KEY = file_key.read()
file_key.close()

quandl.ApiConfig.api_key = API_KEY

spy_tick = 'EOD/SPY'
data = quandl.get(spy_tick, start_date=STARTDATE, end_date=ENDDATE)[['Adj_Close']]
spy = data.resample('M').last().pct_change()
spy.rename(columns={'Adj_Close':'SPY'},inplace=True)
spy.dropna(inplace=True)

#DATE_NOV = pd.to_datetime('2021-11-30')
#spy.loc[DATE_NOV,'SPY'] = rets_raw.loc[DATE_NOV,'SPY']

rf_tick = 'YC/USA3M'
data = quandl.get(rf_tick, start_date=STARTDATE, end_date=ENDDATE)
rf = data.resample('M').last()/(12*100)
rf.rename(columns={'Rate':'US3M'},inplace=True)
rf.dropna(inplace=True)

In [5]:
LEVEL = 'DGS10'
SLOPE = 'T10Y2Y'
INFLATION = 'CORESTICKM159SFRBATL'
macro_raw = pdr.DataReader([LEVEL,SLOPE,INFLATION],data_source='fred', start=STARTDATE, end=ENDDATE)
macro = macro_raw.resample('M').last().dropna()
macro.columns = ['Level','Slope','Inflation']
macro['Inflation Growth'] = macro['Inflation'].diff(12)
macro.drop(columns=['Inflation'],inplace=True)

In [6]:
spy, signals = spy.align(macro,join='inner',axis=0)
rf, _ = rf.align(spy,join='right',axis=0)

# FX Carry

In [7]:
START_DATE = '1999-01-01'
END_DATE = '2021-10-31'
TICKS_FX = ['DEXMXUS']
TICKS_RF = ['USD3MTD156N','IR3TIB01MXM156N']

rfraw = pdr.DataReader(TICKS_RF,data_source='fred', start=START_DATE,end=END_DATE)
fxraw = pdr.DataReader(TICKS_FX,data_source='fred', start=START_DATE,end=END_DATE)

In [8]:
FREQ = 'M'
    
if FREQ == 'M':
    SCALE = 30
elif FREQ == 'Y':
    SCALE = ANNUALIZATION
elif FREQ == 'D':
    SCALE = 1
else:
    error('Frequency needs specified.')

In [9]:
rfFX = pd.DataFrame(index=rfraw.index)
for tick in TICKS_RF:
    lab = tick[0:5]
    rfFX[lab] = rfraw[tick]/100
    
    if 'GBP' in lab:
        ANNUALIZATION = 365
    else:
        ANNUALIZATION = 360        
    
    rfFX[lab] *= (SCALE/ANNUALIZATION)

rfFX = rfFX.resample(FREQ).last()
rfFX.columns = ['RF-USD','RF-MXN']
rfFX

,RF-USD,RF-MXN
DATE,,
1999-01-31,0.004141,0.029542
1999-02-28,0.004189,0.026125
1999-03-31,0.004167,0.021975
1999-04-30,0.004157,0.019158
1999-05-31,0.004222,0.019383
...,...,...
2021-06-30,0.000121,0.003592
2021-07-31,0.000098,0.003817
2021-08-31,0.000100,0.003925


In [10]:
fx = pd.DataFrame(index=fxraw.index)

for tick in TICKS_FX:
    lab = tick[-4:]
    if lab[-2:] == 'US':
        lab = lab[-2:] + lab[-4:-2]
        fx[lab] = 1/fxraw[tick]
    else:
        fx[lab] = fxraw[tick]
    
fx = fx.resample('M').last()
fx

,USMX
DATE,
1999-01-31,0.098503
1999-02-28,0.100639
1999-03-31,0.105031
1999-04-30,0.108196
1999-05-31,0.102119
...,...
2021-06-30,0.050236
2021-07-31,0.050388
2021-08-31,0.049816


# Save the Data

In [11]:
spy.index.name = 'date'
rf.index.name = 'date'
signals.index.name = 'date'
fx.index.name = 'date'
rfFX.index.name = 'date'

In [12]:
with pd.ExcelWriter('final_exam_data.xlsx') as writer:  
#    info.to_excel(writer, sheet_name = 'descriptions')
    signals.to_excel(writer, sheet_name= 'signals')
    spy.to_excel(writer, sheet_name='spy (total returns)')
    fx.to_excel(writer, sheet_name='fx rates')
    rfFX.to_excel(writer, sheet_name='risk-free rates')